Импорт библиотек

In [1]:
import pandas as pd
import requests
import time
import json
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime

# Задачи:
1. ~~Убрать дубликаты матчей~~
2. ~~Перевести время (start_time) и добавить разделитель в duration.~~ ~~Разбить id героев по столбцам~~
3. ~~Поднять PostgreSQL~~ и ~~добавить загрузку~~
4. Автоматизировать (Cron) ~~(windows)~~
5. Убрать при сохранении столбец индекса


Автоматизацию можно настроить через Apache Airflow или через Cron. Для простоты сделал локально через планировщик Windows.

Скорее всего в будущем переделаю схему: выгрузка с API и трансформация во временную таблицу, подключение к БД и insert в основную таблицу

#Выгрузка

In [2]:
# Путь к файлу
filepath = Path('df.csv')

In [3]:
# Загрузка файла
df = pd.read_csv(filepath)

In [4]:
df

,Unnamed: 0,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,0,6833937813,5724027212,True,1667190893,1502,3167.0,1.0,7,22,81,3,185,"136,86,94,11,135","9,71,44,36,119"
1,1,6833934314,5724024879,False,1667190637,1570,NaN,NaN,7,22,81,5,151,"5,88,93,100,17","52,35,38,9,62"
2,2,6833945514,5724035254,False,1667191374,1549,5063.0,5.0,7,3,81,9,227,"28,64,95,101,62","9,19,112,44,39"
3,3,6833938207,5724033627,False,1667190914,1924,4830.0,2.0,7,22,81,3,154,"74,41,105,75,59","25,93,107,120,87"
4,4,6833945600,5724038589,False,1667191395,1797,4382.0,3.0,7,22,81,4,251,"41,97,75,76,44","21,106,70,62,30"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,184,6841367208,5730746790,True,1667504271,1966,3975.0,5.0,7,22,81,8,274,"9,6,57,129,25","5,11,69,72,93"
185,185,6841374802,5730748124,True,1667504548,1785,3621.0,4.0,7,22,81,7,274,"86,100,29,8,31","40,64,47,93,14"
186,186,6841429713,5730756424,True,1667506589,1965,2637.0,1.0,7,22,81,6,274,"91,43,72,129,89","100,9,25,5,10"
187,187,6841429418,5730755862,False,1667506593,1919,6047.0,4.0,7,22,81,6,274,"129,87,13,123,59","137,76,100,35,136"


In [5]:
# Дроп колонки индекса от предыдущего сохранения. !!! Переделать сохранение index=False
df = df.drop(df.columns[[0]], axis=1)
df.head(1)

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,6833937813,5724027212,True,1667190893,1502,3167.0,1.0,7,22,81,3,185,"136,86,94,11,135","9,71,44,36,119"


In [6]:
# API запрос с переводом json'а в датафрейм
j = requests.get(f'https://api.opendota.com/api/publicMatches')
publicMatches = j.json()
df_publicmatches = pd.DataFrame.from_dict(publicMatches)

In [7]:
# Фильтрация высокоранговых матчей. Проверка количетсва строк
df_publicmatches_hight_mmr = df_publicmatches.query('avg_rank_tier == 81')
df_publicmatches_hight_mmr.shape

(3, 14)

In [8]:
# Присоединения новых строк к датафрейму. Проверка количества строк
frames = [df, df_publicmatches_hight_mmr]
df = pd.concat(frames)
df.shape

(192, 14)

In [9]:
# Создание массива и датафрейма с уникальными match_id для более сложного API запроса с более полной статистикой по матчу. Требует доработки
# matches_array = df.match_id.unique()
# matches_array = pd.DataFrame.from_dict(matches_array)
# df_match_id = matches_array.rename(columns={ 0 : "match_id"})
# df_match_id.shape

In [10]:
# Так как API запрос парсит недавние 100 матчей, рандомно. То могут встречаться дубликаты. Избавимся от них
df = df.drop_duplicates(subset=['match_id'])

In [11]:
# Сохранение 
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)

#Трансформация

In [12]:
# Перевод в формат времени
# df['start_time'] = pd.to_datetime(df['start_time'], unit='s')
# df['duration'] = pd.to_datetime(df['duration'], unit="s").dt.time

In [13]:
# Сплит колонок
# df[['dtp_1', 'dtp_2', 'dtp_3', 'dtp_4', 'dtp_5']] = df['dire_team']. str.split(',' , expand= True )
# df[['rtp_1', 'rtp_2', 'rtp_3', 'rtp_4', 'rtp_5']] = df['radiant_team']. str.split(',' , expand= True )

# Загрузка

In [14]:
# Подключение к БД
# from sqlalchemy import create_engine
# con = create_engine()

In [15]:
# Ускорение выгрузки
# import csv
# from io import StringIO

# def psql_insert_copy(table, conn, keys, data_iter):
#     # gets a DBAPI connection that can provide a cursor
#     dbapi_conn = conn.connection
#     with dbapi_conn.cursor() as cur:
#         s_buf = StringIO()
#         writer = csv.writer(s_buf)
#         writer.writerows(data_iter)
#         s_buf.seek(0)

#         columns = ', '.join('"{}"'.format(k) for k in keys)
#         if table.schema:
#             table_name = '{}.{}'.format(table.schema, table.name)
#         else:
#             table_name = table.name

#         sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
#             table_name, columns)
#         cur.copy_expert(sql=sql, file=s_buf)

In [16]:
# df.to_sql('matches_stat', con, index=False, if_exists='replace', method=psql_insert_copy)